# Gemini Plugin

> Plugin implementation for Google Gemini API transcription

In [ ]:
#| default_exp plugin

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import logging
import tempfile
from pathlib import Path
from typing import Dict, Any, Optional, List, Union, Tuple
import re

import numpy as np

try:
    from google import genai
    from google.genai import types
    GEMINI_AVAILABLE = True
except ImportError:
    GEMINI_AVAILABLE = False

try:
    from cjm_ffmpeg_utils.core import FFMPEG_AVAILABLE
    from cjm_ffmpeg_utils.audio import downsample_audio
except ImportError:
    FFMPEG_AVAILABLE = False
    
from cjm_transcription_plugin_system.plugin_interface import PluginInterface
from cjm_transcription_plugin_system.core import AudioData, TranscriptionResult

In [ ]:
#| export
class GeminiPlugin(PluginInterface):
    """Google Gemini API transcription plugin."""
    
    # Default audio-capable models (can be overridden)
    DEFAULT_AUDIO_MODELS = [
        "gemini-2.5-flash",
        "gemini-2.5-flash-preview-05-20",
        "gemini-2.5-pro",
        "gemini-2.5-pro-preview-05-06",
        "gemini-2.0-flash",
        "gemini-2.0-flash-exp",
        "gemini-1.5-flash",
        "gemini-1.5-flash-latest",
        "gemini-1.5-pro",
        "gemini-1.5-pro-latest",
    ]
    
    def __init__(self):
        """Initialize the Gemini plugin with default configuration."""
        self.logger = logging.getLogger(f"{__name__}.{type(self).__name__}")
        self.config = {}
        self.client = None
        self.available_models = []
    
    @property
    def name(
        self
    ) -> str:  # Returns the plugin name identifier
        """Return the plugin name identifier."""
        return "gemini"
    
    @property
    def version(
        self
    ) -> str:  # Returns the plugin version string
        """Return the plugin version string."""
        return "1.0.0"
    
    @property
    def supported_formats(
        self
    ) -> List[str]:  # Returns list of supported audio formats
        """Return list of supported audio file formats."""
        return ["wav", "mp3", "aiff", "aac", "ogg", "flac"]
    
    def get_config_schema(
        self
    ) -> Dict[str, Any]:  # Returns JSON schema for configuration validation
        """Return configuration schema for Gemini."""
        return {
            "$schema": "http://json-schema.org/draft-07/schema#",
            "type": "object",
            "title": "Gemini Configuration",
            "properties": {
                "model": {
                    "type": "string",
                    "default": "gemini-2.5-flash",
                    "description": "Gemini model to use for transcription",
                    "enum": self.DEFAULT_AUDIO_MODELS  # Will be updated dynamically
                },
                "api_key": {
                    "type": ["string", "null"],
                    "default": None,
                    "description": "Google API key (defaults to GEMINI_API_KEY env var)"
                },
                "prompt": {
                    "type": "string",
                    "default": "Generate a transcription of the audio, only extract speech and ignore background audio.",
                    "description": "Prompt for transcription"
                },
                "temperature": {
                    "type": "number",
                    "minimum": 0.0,
                    "maximum": 2.0,
                    "default": 0.0,
                    "description": "Sampling temperature"
                },
                "top_p": {
                    "type": "number",
                    "minimum": 0.0,
                    "maximum": 1.0,
                    "default": 0.95,
                    "description": "Top-p sampling parameter"
                },
                "max_output_tokens": {
                    "type": "integer",
                    "minimum": 1,
                    "maximum": 65536,
                    "default": 65536,
                    "description": "Maximum number of output tokens"
                },
                "seed": {
                    "type": ["integer", "null"],
                    "default": None,
                    "description": "Random seed for reproducibility"
                },
                "response_mime_type": {
                    "type": "string",
                    "enum": ["text/plain", "application/json"],
                    "default": "text/plain",
                    "description": "Response MIME type"
                },
                "downsample_audio": {
                    "type": "boolean",
                    "default": False,
                    "description": "Downsample audio before uploading (requires ffmpeg)"
                },
                "downsample_rate": {
                    "type": "integer",
                    "enum": [8000, 16000, 22050, 44100],
                    "default": 16000,
                    "description": "Target sample rate for downsampling"
                },
                "downsample_channels": {
                    "type": "integer",
                    "enum": [1, 2],
                    "default": 1,
                    "description": "Number of audio channels (1=mono, 2=stereo)"
                },
                "safety_settings": {
                    "type": "string",
                    "enum": ["OFF", "BLOCK_NONE", "BLOCK_FEW", "BLOCK_SOME", "BLOCK_MOST"],
                    "default": "OFF",
                    "description": "Safety filter threshold"
                },
                "auto_refresh_models": {
                    "type": "boolean",
                    "default": True,
                    "description": "Automatically refresh available models list"
                },
                "model_filter": {
                    "type": "array",
                    "items": {"type": "string"},
                    "default": [],
                    "description": "Keywords to exclude from model names (e.g., ['tts', 'image'])"
                }
            },
            "required": ["model"],
            "additionalProperties": False
        }
    
    def get_current_config(
        self
    ) -> Dict[str, Any]:  # Returns the merged configuration dictionary
        """Return current configuration."""
        defaults = self.get_config_defaults()
        return {**defaults, **self.config}
    
    def _get_api_key(
        self
    ) -> str:  # Returns the API key string
        """Get API key from config or environment."""
        api_key = self.config.get("api_key")
        if not api_key:
            api_key = os.environ.get("GEMINI_API_KEY")
        if not api_key:
            raise ValueError("No API key provided. Set GEMINI_API_KEY environment variable or provide api_key in config")
        return api_key
    
    def _refresh_available_models(
        self
    ) -> List[str]:  # Returns list of available model names
        """Fetch and filter available models from Gemini API."""
        try:
            if not self.client:
                return self.DEFAULT_AUDIO_MODELS
            
            # Get all models that support content generation
            all_models = list(self.client.models.list())
            gen_models = [model for model in all_models if 'generateContent' in model.supported_actions]
            
            # Extract model names and apply filters
            model_filter = self.config.get("model_filter", [])
            if not model_filter:
                model_filter = ['tts', 'image', 'learn']  # Default exclusions
            
            filtered_names = []
            for model in gen_models:
                model_name = model.name.removeprefix('models/')
                # Skip if any filter keyword is in the model name
                if not any(keyword in model_name.lower() for keyword in model_filter):
                    filtered_names.append(model_name)
            
            # Sort with newest/best models first
            filtered_names.sort(reverse=True)
            
            self.logger.info(f"Found {len(filtered_names)} audio-capable models")
            return filtered_names if filtered_names else self.DEFAULT_AUDIO_MODELS
            
        except Exception as e:
            self.logger.warning(f"Could not fetch models from API: {e}. Using defaults.")
            return self.DEFAULT_AUDIO_MODELS
    
    def initialize(
        self,
        config: Optional[Dict[str, Any]] = None  # Configuration dictionary to override defaults
    ) -> None:
        """Initialize the plugin with configuration."""
        if config:
            is_valid, error = self.validate_config(config)
            if not is_valid:
                raise ValueError(f"Invalid configuration: {error}")
        
        # Merge with defaults
        defaults = self.get_config_defaults()
        self.config = {**defaults, **(config or {})}
        
        # Initialize client
        try:
            api_key = self._get_api_key()
            self.client = genai.Client(api_key=api_key)
            
            # Refresh available models if enabled
            if self.config.get("auto_refresh_models", True):
                self.available_models = self._refresh_available_models()
                # Update schema with actual available models
                schema = self.get_config_schema()
                schema["properties"]["model"]["enum"] = self.available_models
            else:
                self.available_models = self.DEFAULT_AUDIO_MODELS
                
            self.logger.info(f"Initialized Gemini plugin with model '{self.config['model']}'")
            
        except Exception as e:
            self.logger.error(f"Failed to initialize Gemini client: {e}")
            raise
    
    def _prepare_audio(
        self,
        audio: Union[AudioData, str, Path]  # Audio data object or path to audio file
    ) -> Tuple[Path, bool]:  # Returns tuple of (processed audio path, whether temp file was created)
        """Prepare audio file for upload.
        
        Returns:
            Tuple of (audio_path, is_temp_file)
        """
        temp_created = False
        
        if isinstance(audio, (str, Path)):
            audio_path = Path(audio)
        elif isinstance(audio, AudioData):
            # Save AudioData to temporary file
            import soundfile as sf
            temp_file = tempfile.NamedTemporaryFile(suffix=".wav", delete=False)
            
            audio_array = audio.samples
            # Convert to mono if stereo
            if audio_array.ndim > 1:
                audio_array = audio_array.mean(axis=1)
            
            # Ensure float32 and normalized
            if audio_array.dtype != np.float32:
                audio_array = audio_array.astype(np.float32)
            if audio_array.max() > 1.0:
                audio_array = audio_array / np.abs(audio_array).max()
            
            sf.write(temp_file.name, audio_array, audio.sample_rate)
            audio_path = Path(temp_file.name)
            temp_created = True
        else:
            raise ValueError(f"Unsupported audio input type: {type(audio)}")
        
        # Optionally downsample audio
        if self.config.get("downsample_audio", False) and FFMPEG_AVAILABLE:
            try:
                downsampled = audio_path.with_stem(f"{audio_path.stem}_downsampled")
                downsample_audio(
                    audio_path,
                    downsampled,
                    # sample_rate=self.config.get("downsample_rate", 16000),
                    # channels=self.config.get("downsample_channels", 1)
                )
                
                # Clean up original temp file if created
                if temp_created:
                    audio_path.unlink()
                
                audio_path = downsampled
                temp_created = True
                self.logger.info(f"Downsampled audio to {self.config['downsample_rate']}Hz")
                
            except Exception as e:
                self.logger.warning(f"Failed to downsample audio: {e}")
        
        return audio_path, temp_created
    
    def execute(
        self,
        audio: Union[AudioData, str, Path],  # Audio data object or path to audio file
        **kwargs # Additional arguments to override config
    ) -> TranscriptionResult:  # Returns transcription result object
        """Transcribe audio using Gemini."""
        if not self.client:
            raise RuntimeError("Plugin not initialized. Call initialize() first.")
        
        # Prepare audio file
        audio_path, temp_created = self._prepare_audio(audio)
        
        try:
            # Merge runtime kwargs with config
            exec_config = {**self.config, **kwargs}
            
            # Read audio file
            with open(audio_path, 'rb') as f:
                audio_bytes = f.read()
            
            # Determine MIME type
            suffix = audio_path.suffix.lower()
            mime_map = {
                '.wav': 'audio/wav',
                '.mp3': 'audio/mp3',
                '.aiff': 'audio/aiff',
                '.aac': 'audio/aac',
                '.ogg': 'audio/ogg',
                '.flac': 'audio/flac'
            }
            mime_type = mime_map.get(suffix, 'audio/wav')
            
            # Create audio content part
            audio_content = types.Part.from_bytes(
                data=audio_bytes,
                mime_type=mime_type
            )
            
            # Prepare generation config
            generate_config = types.GenerateContentConfig(
                response_mime_type=exec_config["response_mime_type"],
                temperature=exec_config["temperature"],
                top_p=exec_config["top_p"],
                max_output_tokens=exec_config["max_output_tokens"],
                seed=exec_config.get("seed"),
                safety_settings=[
                    types.SafetySetting(
                        category="HARM_CATEGORY_HATE_SPEECH",
                        threshold=exec_config["safety_settings"]
                    ),
                    types.SafetySetting(
                        category="HARM_CATEGORY_DANGEROUS_CONTENT",
                        threshold=exec_config["safety_settings"]
                    ),
                    types.SafetySetting(
                        category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
                        threshold=exec_config["safety_settings"]
                    ),
                    types.SafetySetting(
                        category="HARM_CATEGORY_HARASSMENT",
                        threshold=exec_config["safety_settings"]
                    )
                ]
            )
            
            # Prepare contents
            prompt = exec_config["prompt"]
            contents = [prompt, audio_content]
            
            # Generate transcription
            self.logger.info(f"Transcribing with Gemini model: {exec_config['model']}")
            
            response = self.client.models.generate_content(
                model=exec_config["model"],
                contents=contents,
                config=generate_config
            )
            
            # Extract text from response
            transcribed_text = response.text if hasattr(response, 'text') else str(response)
            
            # Create result
            result = TranscriptionResult(
                text=transcribed_text.strip(),
                confidence=None,  # Gemini doesn't provide confidence scores
                segments=None,  # Gemini doesn't provide segments by default
                metadata={
                    "model": exec_config["model"],
                    "temperature": exec_config["temperature"],
                    "top_p": exec_config["top_p"],
                    "prompt": prompt[:100] + "..." if len(prompt) > 100 else prompt
                }
            )
            
            self.logger.info(f"Transcription completed: {len(transcribed_text.split())} words")
            return result
            
        finally:
            # Clean up temporary file
            if temp_created:
                try:
                    audio_path.unlink()
                except Exception:
                    pass
    
    def is_available(
        self
    ) -> bool:  # Returns True if the Gemini API is available
        """Check if Gemini API is available."""
        return GEMINI_AVAILABLE
    
    def cleanup(
        self
    ) -> None:
        """Clean up resources."""
        self.client = None
        self.logger.info("Cleanup completed")
    
    def get_available_models(
        self
    ) -> List[str]:  # Returns list of available model names
        """Get list of available audio-capable models."""
        if self.config.get("auto_refresh_models", True) and self.client:
            self.available_models = self._refresh_available_models()
        return self.available_models

## Testing the Plugin

In [ ]:
# Test basic functionality
plugin = GeminiPlugin()

# Check availability
print(f"Gemini available: {plugin.is_available()}")
print(f"Plugin name: {plugin.name}")
print(f"Plugin version: {plugin.version}")
print(f"Supported formats: {plugin.supported_formats}")

Gemini available: True
Plugin name: gemini
Plugin version: 1.0.0
Supported formats: ['wav', 'mp3', 'aiff', 'aac', 'ogg', 'flac']


In [ ]:
# Test configuration schema
schema = plugin.get_config_schema()
print("Configuration properties:")
for prop, details in list(schema["properties"].items())[:5]:
    print(f"  {prop}: {details.get('description', 'No description')}")

Configuration properties:
  model: Gemini model to use for transcription
  api_key: Google API key (defaults to GEMINI_API_KEY env var)
  prompt: Prompt for transcription
  temperature: Sampling temperature
  top_p: Top-p sampling parameter


In [ ]:
# Test initialization (requires API key)
if os.environ.get("GEMINI_API_KEY"):
    plugin.initialize({"model": "gemini-2.5-flash"})
    print(f"Initialized with model: {plugin.config['model']}")
    
    # Get available models
    models = plugin.get_available_models()
    print(f"\nFound {len(models)} available models")
    print("Top 5 models:")
    for model in models[:5]:
        print(f"  - {model}")
else:
    print("Set GEMINI_API_KEY environment variable to test initialization")

Initialized with model: gemini-2.5-flash

Found 36 available models
Top 5 models:
  - gemma-3n-e4b-it
  - gemma-3n-e2b-it
  - gemma-3-4b-it
  - gemma-3-27b-it
  - gemma-3-1b-it


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()